In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [2]:
# Read product groups file
product_groups = pd.read_csv("../data/product_groups.csv")
product_groups = product_groups.astype(object)
product_groups

,category_number,category_level_1,category_level_2,category_level_3,category_level_4
0,9046,341,20,10,10
1,9009,125,25,10,10
2,9049,410,65,20,10
3,9019,300,10,19,11
4,9035,305,31,10,10
...,...,...,...,...,...
3908,9019,616,95,20,15
3909,9020,616,95,20,10
3910,9020,616,95,20,15
3911,9020,616,95,20,30


In [90]:
# Check if category_level1, level2, level3, level4 -> category_number is a function (mathematical):

product_groups["hash"] = product_groups["category_level_1"].astype(str) + (product_groups["category_level_2"].astype(str))
hash_to_category_number = {}
category_number_to_hash = {}


# Keep hash as the {[first category level][second_category_level[0]]}
for index, row in product_groups[["category_number", "hash"]].iterrows():
    hash = row[1]
    if hash not in hash_to_category_number:
        hash_to_category_number[hash] = []

    if row["category_number"] not in hash_to_category_number[hash]:
        hash_to_category_number[hash].append(row["category_number"])

sum = 0
for hash in hash_to_category_number:
    if len(hash_to_category_number[hash]) > 1:
        sum += 1

for row in product_groups[["category_number", "hash"]].itertuples():
    hash = row[2]
    category_number = row[1]
    if category_number not in category_number_to_hash:
        category_number_to_hash[category_number] = []
    
    if hash not in category_number_to_hash[category_number]:
        category_number_to_hash[category_number].append(hash)
    
#for category_number in category_number_to_hash:
#    print(f"{category_number}: {category_number_to_hash[category_number]}")

for hash in hash_to_category_number:
    print(f"{hash}: {hash_to_category_number[hash]}")

34120: [9046]
12525: [9009]
41065: [9049]
30010: [9019, 9035]
30531: [9035]
33575: [9044]
31311: [9056]
42545: [9057]
61630: [9052]
30521: [9035]
61640: [9055]
63641: [9038]
66040: [9054]
22045: [9032]
11010: [9001]
61335: [9011]
62825: [9030]
67040: [9053]
60325: [9000]
41050: [9049]
33561: [9041, 9044]
41025: [9049]
65055: [9044]
31040: [9038]
22060: [9060]
42510: [9049]
41035: [9049]
30510: [9035]
64615: [9042]
31316: [9056]
31160: [9037]
13130: [9006]
30020: [9035]
60950: [9001, 9058]
33555: [9044]
64430: [9037]
61755: [9058, 9010]
60330: [9000, 9006]
42075: [9052]
65760: [9049]
34040: [9046]
61765: [9012, 9015]
65735: [9049]
10325: [9000]
20015: [9022, 9059]
33560: [9044]
30023: [9035]
65715: [9049]
12550: [9012, 9006, 9002, 9008, 9007, 9044, 9005]
41040: [9049]
66545: [9057]
31210: [9040]
42550: [9057, 9049]
33015: [9043]
65060: [9044]
33550: [9044]
21020: [9028, 9021, 9022, 9024, 9059, 9032]
30061: [9000]
42055: [9057]
65755: [9049]
64610: [9042]
40065: [9048]
65450: [9048]
2002

RESULTS:

There are 151 hashes pointing to at least 2 different categories.

There are 010 hashes pointing to at least 3 different categories.

There are 002 hashes pointing to at least 4 different categories.


In [4]:
# Get card number - individual number match:

shopping_by_card_number = pd.read_csv("../data/transaction_header.csv")
shopping_by_card_number = shopping_by_card_number.drop(columns="date_of_transaction")
shopping_by_card_number = shopping_by_card_number.astype(object)
print(shopping_by_card_number.describe()) # Each basket id is unique 
print()

cardnumber_to_individualnumber = pd.read_csv("../data/customeraccount.csv")
#cardnumber_to_individualnumber.index = cardnumber_to_individualnumber["cardnumber"]

cardnumber_to_individualnumber = cardnumber_to_individualnumber.astype(object)
print(cardnumber_to_individualnumber.describe()) # Card number is unique,
# But some individuals have more than one cards: 28593 individuals, 35159 cards.
#cardnumber_to_individualnumber.drop(columns="cardnumber", inplace=True)
cardnumber_to_individualnumber

              cardnumber           basketid  is_sanal
count            1124673            1124673   1124673
unique             30319            1124673         2
top     9887854616568815  20120163990041163         0
freq                1239                  1   1024395

        individualnumber        cardnumber
count              35159             35159
unique             28593             35159
top             96035559  7287134635560315
freq                  49                 1


,individualnumber,cardnumber
0,16481068,7287134635560315
1,17634947,6587004647560415
2,19323290,8187034648564315
3,19323290,3005502487247749
4,37820213,6287114610560316
...,...,...
35154,100998656,2687784602565315
35155,101074070,6387894671560235
35156,101270229,9287054697566815
35157,101328888,2587064608564915


In [5]:
# Get basket id - individual number match

shopping_by_individual_number = pd.merge(shopping_by_card_number,cardnumber_to_individualnumber,how='inner',on='cardnumber')
shopping_by_individual_number = shopping_by_individual_number[["individualnumber", "basketid", "is_sanal"]]
shopping_by_individual_number.astype(object).describe()

basket_id_to_individual_number = shopping_by_individual_number[["basketid", "individualnumber"]].set_index("basketid")
basket_id_to_individual_number = basket_id_to_individual_number["individualnumber"].to_dict()
basket_id_to_individual_number

{20120163990041163: 68318409,
 20120436020022952: 68318409,
 20121763990012494: 68318409,
 20121810750037221: 68318409,
 20122963990049952: 68318409,
 21010263990056580: 68318409,
 21010563990057591: 68318409,
 21011663990034442: 68318409,
 21011851920010530: 68318409,
 21012210750033935: 68318409,
 21012430120049245: 68318409,
 21012563990036782: 68318409,
 21012610750034886: 68318409,
 21012710750024160: 68318409,
 21012763130024231: 68318409,
 21012863130034530: 68318409,
 21013063990017023: 68318409,
 21020263990040918: 68318409,
 21020336020030297: 68318409,
 21020363990039331: 68318409,
 21020710750016156: 68318409,
 21021063990010493: 68318409,
 21021063990025962: 68318409,
 21021436020058123: 68318409,
 21021610750030182: 68318409,
 21021636020033594: 68318409,
 21021610750030183: 68318409,
 21021963990013724: 68318409,
 21021963990034471: 68318409,
 21022063990047110: 68318409,
 21022010750031218: 68318409,
 21022330120040989: 68318409,
 21030210750033836: 68318409,
 210306639

In [6]:
transaction_sale = pd.read_csv("../data/transaction_sale/transaction_sale.csv")
transaction_sale.astype(object).describe()

,basketid,category_level_1,category_level_2,category_level_3,category_level_4,amount,quantity,discount_type_1,discount_type_2,discount_type_3
count,6537881,6537881,6537881,6537881,6537881,6537881.00,6537881.0,1889008.0,6537751.0,6537753.0
unique,1124327,70,40,40,85,24700.00,8380.0,1557194.0,5887.0,6863.0
top,21050512030076004,130,10,10,10,5.95,1.0,0.0,0.0,0.0
freq,165,872712,1376177,2561787,2220686,86873.00,4290925.0,36184.0,5939946.0,5628181.0


* Implemented three methods for storing the shopping history of a customer.
* Discount types are not taken into consideration.

### Store Type 1: _Store each buy seperately_

In this format data is stored in the following way:

    {
        "individual_number_1": [
            {"category_1": 130, "category_2": 45, "category_3": 10, "category_4": 20, "amount": 19.90, "quantity": 1.000}
            {"category_1": 150, "category_2": 15, "category_3": 15, "category_4": 25, "amount": 3.50, "quantity": 1.430}
        ]
        "individual_number_2": [
            {"category_1": 617, "category_2": 25, "category_3": 15, "category_4": 10, "amount": 8.75, "quantity": 2.000}
        ]
    }

### Store Type 2: _Store by total volume of the buy, classify them according to category level 1 and category level 2_

In this format data is stored in the following way:

    {
        "individual_number_1": {

            613: {
                50: [3.0, 49.75], 
                40: [11.0, 248.65], 
                15: [2.0, 8.9]
            }, 

            425: {
                30: [2.0, 11.4]
            }, 
            
            130: {
                85: [10.0, 165.06]
            }
        }
    }

* First key represents the category level 1.
* Second key represents the category level 2.
* Inner list stores the quantity and total money spent on that combination of category_1, category_2:
* [quantity, total_amount]

### Store Type 3: _Store by total volume of the buy, classify them according to category number used in campaigns_

In this format data is stored in the following way:

    {
        "individual_number_1": {

            9049: [47.89, 3054.63],
            9013: [xx.xx, xxxx.xx],
            ...
            "unknown": [xxx.xx, xxxxx.xx]
        }
        
        "individual_number_2": {

            9049: [16.23, 2045.82],
            9013: [xx.xx, xxxx.xx],
            ...
            "unknown": [xxx.xx, xxxxx.xx]
        }        
    }

* Category level's are converted to category number used in campaigns by using category_level_1 and category_level_2.
* This method may further change.

In [96]:
# To be fixed:
    # Round quantity and total price data

# Store type 1: ~90s to execute.
# Store type 2: ~25s to execute.
# Store type 3: ~30s to execute.

# Select store type: 1 or 2 or 3: (0 for to see individual_shopping_volume only)
store_type = 3

individual_shopping_information = {}
individual_shopping_volume = {} # Simple dictionary: {"individual_number": total_money_spent}

# Read the transaction_sale file and save each customer's shopping history in the "individual_shopping_information" dictionary.
for row in transaction_sale.itertuples(): 
    # Get necessary features:

    basket_id = row[1]
    category_1 = row[2]
    category_2 = row[3]
    category_3 = row[4]
    category_4 = row[5]
    amount = row[6]
    quantity = row[7]
    # discount_1 = row[7]
    # discount_2 = row[8]
    # discount_3 = row[9]
    

    # Get individual number:
    individual_number = basket_id_to_individual_number[basket_id]
    
    if individual_number not in individual_shopping_volume:
        individual_shopping_volume[individual_number] = 0
    individual_shopping_volume[individual_number] += quantity * amount

    if store_type == 1:
        # Storing method: type1 -> Store the products seperately.
        # Initialize customer shopping information if customer is not in information_shopping_information:
        if individual_number not in individual_shopping_information:
            individual_shopping_information[individual_number] = []
            
        product_information = {}
        product_information["category_level_1"] = category_1
        product_information["category_level_2"] = category_2
        product_information["category_level_3"] = category_3
        product_information["category_level_4"] = category_4
        product_information["amount"] = amount
        product_information["quantity"] = quantity

        individual_shopping_information[individual_number].append(product_information)
    
    elif store_type == 2:
        # Storing method: type2 -> Store the products by keys as category_1 and category_2, values = volume of the buy history.
        # Initialize customer shopping information if customer is not in individual_shopping_information:        
        if individual_number not in individual_shopping_information:
            individual_shopping_information[individual_number] = {}
        
        # Initialize category_1 in customer's shopping history if category_1 is not in customer's shopping history:
        if category_1 not in individual_shopping_information[individual_number]:
            individual_shopping_information[individual_number][category_1] = {}

        # Initialize category_1 in customer's category_1 shopping history if category_2 is not in customer's category_1 shopping history:
        if category_2 not in individual_shopping_information[individual_number][category_1]:
             # 0th index will be quantity, 1st index will be total money spent on that category combination.
            individual_shopping_information[individual_number][category_1][category_2] = [0, 0]
        
        # Increment the total volume in customer's shopping history branch: category_1, branch: category_2:
        individual_shopping_information[individual_number][category_1][category_2][0] += quantity
        individual_shopping_information[individual_number][category_1][category_2][1] += quantity * amount

    elif store_type == 3:
        # Storing method: type3 -> Store the products' volume by category_number used in campaign.
        # To match a product's category level to category type used in campaign, use the hash= {category_level_1 + category_level_2}
        # Initialize customer shopping information if customer is not in individual_shopping_information:
        if individual_number not in individual_shopping_information:
            individual_shopping_information[individual_number] = {}
        
        category_hash = str(category_1) + str(category_2)
        try:
            category_number_list = hash_to_category_number[category_hash]
        except:
            category_number_list = ["unknown"]

        # Initialize category_number in customer's shopping history if category_number is not in customer's shopping history:
        for category_number in category_number_list:
            if category_number not in individual_shopping_information[individual_number]:
                # 0th index will be quantity, 1st index will be total money spent on that category number.
                individual_shopping_information[individual_number][category_number] = [0, 0]
            
            individual_shopping_information[individual_number][category_number][0] += quantity
            individual_shopping_information[individual_number][category_number][1] += quantity * amount

len(individual_shopping_volume)

    

28576

In [99]:
individual_shopping_volume

{90065509: 18246.425990000003,
 56076820: 6791.47322999998,
 37752547: 20023.03321000005,
 98384331: 20207.876070000024,
 94255308: 12685.235679999994,
 93937111: 56137.69781999999,
 94688545: 2658.9989899999996,
 91747483: 22121.39369000003,
 94778614: 34754.819130000054,
 16893075: 5897.690279999993,
 94616834: 14983.75664999999,
 94697868: 1781.6929200000013,
 74174356: 2071.0557500000014,
 96249709: 25915.083099999993,
 73870110: 18599.4210600001,
 16472172: 25518.61192000013,
 88801604: 24991.587529999935,
 93449464: 21185.29941000004,
 92876940: 114220.04547999887,
 89216880: 11175.61615,
 93350998: 8083.432879999992,
 96787839: 33090.33146000021,
 95993648: 2193.0842300000004,
 25235949: 10951.806409999992,
 93862335: 515.19885,
 57167031: 647.40269,
 93318609: 8628.42618999999,
 98243967: 894.92872,
 74945395: 6037.0145799999955,
 68952827: 150.48232,
 93045592: 10389.542840000015,
 97109866: 4959.456939999991,
 93522155: 10824.893220000004,
 97690886: 14307.892350000018,
 9587